# Modèle de base (TF-IDF)

Sommaire : 
1. Importation des packages
2. Importation des données
3. Nettoyage des textes
4. Vectorisation TF-IDF
5. Sélection des labels (multi-label classification)
6. Séparation en train/test
7. Entraînement du modèle baseline (OneVsRest Logistic Regression)
8. Prédictions et évaluation
9. Test sur le fichier de test.csv

## Importation des packages

In [19]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

## 1. Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [ ]:
# Importez le module de lecteur de Google.colab pour interagir avec Google Drive
from google.colab import drive

# Montez Google Drive vers le répertoire '/Content/Drive'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Importation des données d'entraînement à partir de Google Drive
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')

# Affichage des premières lignes du DataFrame pour vérifier l'importation
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## 2. Nettoyage des textes

In [ ]:
# fonction nettoyage du texte
def clean_text(text):
    text = text.lower()  # Minuscule
    text = re.sub(r'\W+', ' ', text)  # Supprimer caractères spéciaux
    return text

# Appliquer la fonction de nettoyage à la colonne 'comment_text' du DataFrame
data['comment_text'] = data['comment_text'].astype(str).apply(clean_text)

## 3. Vectorisation TF-IDF

In [ ]:
# Initialiser le vectoriseur TF-IDF avec un nombre maximum de features
vectorizer = TfidfVectorizer(max_features=10000)  # Limiter le nombre de features

# Appliquer le vectoriseur TF-IDF aux textes nettoyés pour obtenir la matrice de features
X = vectorizer.fit_transform(data['comment_text'])

## 4. Sélection des labels (multi-label classification)

In [ ]:
# Sélectionner les colonnes de labels pour la classification multi-label
y = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

## 5. Séparation en train/test

In [ ]:
# Séparer les données en ensembles d'entraînement et de test
# X : les features (matrice TF-IDF)
# y : les labels (multi-labels)
# test_size=0.2 : 20% des données seront utilisées pour le test, 80% pour l'entraînement
# random_state=42 : pour assurer la reproductibilité des résultats
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 6. Entraînement du modèle baseline (OneVsRest Logistic Regression)

In [ ]:
# Initialiser le modèle OneVsRest avec une régression logistique comme classificateur de base
model = OneVsRestClassifier(LogisticRegression())

# Entraîner le modèle sur les données d'entraînement
# X_train : les features d'entraînement (matrice TF-IDF)
# y_train : les labels d'entraînement (multi-labels)
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

## 7. Prédictions et évaluation

In [ ]:
# Utiliser le modèle entraîné pour faire des prédictions sur les données de test
y_pred = model.predict(X_test)

# Afficher un rapport de classification pour évaluer les performances du modèle
# y_test : les vrais labels des données de test
# y_pred : les labels prédits par le modèle
# target_names=y.columns : les noms des labels pour une meilleure lisibilité du rapport
print(classification_report(y_test, y_pred, target_names=y.columns))

               precision    recall  f1-score   support

        toxic       0.91      0.62      0.74      3056
 severe_toxic       0.59      0.26      0.36       321
      obscene       0.92      0.62      0.74      1715
       threat       0.45      0.12      0.19        74
       insult       0.84      0.53      0.65      1614
identity_hate       0.68      0.15      0.25       294

    micro avg       0.88      0.56      0.68      7074
    macro avg       0.73      0.38      0.49      7074
 weighted avg       0.87      0.56      0.68      7074
  samples avg       0.06      0.05      0.05      7074



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 8. Test sur le fichier de test.csv

In [29]:
# Load and preprocess test data
test_data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test.csv')
test_data['comment_text'] = test_data['comment_text'].astype(str).apply(clean_text)

# Apply TF-IDF vectorization to the test data
X_test_final = vectorizer.transform(test_data['comment_text'])

test_labels = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test_labels.csv')
valid_idx = (test_labels.iloc[:, 1:] != -1).all(axis=1)  # Garder seulement les lignes avec labels valides

# Accessing X_test_final with .tocsc() as it is sparse matrix.
X_test_valid = X_test_final.tocsc()[valid_idx]
y_test_valid = test_labels.iloc[:, 1:][valid_idx]

y_pred_valid = model.predict(X_test_valid)
print(classification_report(y_test_valid, y_pred_valid, target_names=y.columns))

               precision    recall  f1-score   support

        toxic       0.66      0.69      0.67      6090
 severe_toxic       0.40      0.31      0.35       367
      obscene       0.79      0.59      0.67      3691
       threat       0.48      0.18      0.27       211
       insult       0.76      0.50      0.60      3427
identity_hate       0.70      0.24      0.35       712

    micro avg       0.70      0.58      0.63     14498
    macro avg       0.63      0.42      0.49     14498
 weighted avg       0.71      0.58      0.63     14498
  samples avg       0.06      0.05      0.05     14498



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
